In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [9]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [10]:
val_split = 0.2
val_size = int(val_split * len(x_train))

x_val = x_train[-val_size:]
y_val = y_train[-val_size:]

x_train = x_train[:-val_size]
y_train = y_train[:-val_size]

In [11]:
model = keras.Sequential([
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(256, kernel_size=(3, 3), activation="relu"),
    layers.Flatten(),
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

In [12]:
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomTranslation(0.1, 0.1)
])
model = keras.Sequential([
    data_augmentation,
    model
])

In [13]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [14]:
epochs = 5

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True
)

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=epochs,
    callbacks=[early_stopping]
)


Epoch 1/5


D:\Anaconda\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1500/1500 [==============================] - 113s 69ms/step - loss: 0.7473 - accuracy: 0.7140 - val_loss: 0.6708 - val_accuracy: 0.7473
Epoch 2/5
1500/1500 [==============================] - 105s 70ms/step - loss: 0.5386 - accuracy: 0.7963 - val_loss: 0.5188 - val_accuracy: 0.8112
Epoch 3/5
1500/1500 [==============================] - 104s 69ms/step - loss: 0.4795 - accuracy: 0.8181 - val_loss: 0.4613 - val_accuracy: 0.8324
Epoch 4/5
1500/1500 [==============================] - 104s 69ms/step - loss: 0.4379 - accuracy: 0.8338 - val_loss: 0.3994 - val_accuracy: 0.8533
Epoch 5/5
1500/1500 [==============================] - 103s 69ms/step - loss: 0.4099 - accuracy: 0.8463 - val_loss: 0.4516 - val_accuracy: 0.8324


In [15]:
train_acc = model.evaluate(x_train, y_train)
print("Train accuracy:", train_acc)

1500/1500 [==============================] - 9s 6ms/step - loss: 0.4147 - accuracy: 0.8417
Train accuracy: [0.41470712423324585, 0.8417083621025085]


In [16]:
test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)

313/313 [==============================] - 2s 6ms/step - loss: 0.4584 - accuracy: 0.8297
Test accuracy: [0.4583583474159241, 0.8296999931335449]


In [17]:
import pickle


with open("fashion_mnist_model.pkl", "wb") as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://217ab0ab-13d0-4672-bcea-e369e0bb49e9/assets


INFO:tensorflow:Assets written to: ram://217ab0ab-13d0-4672-bcea-e369e0bb49e9/assets
